<a href="https://colab.research.google.com/github/maryvorob/colab/blob/master/%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%A2%D0%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**1. Какая разница между двойными и одинарными кавычками в SQL?**

> Двойные кавычки предназначены для названий (имен таблиц или полей).
Одиночные кавычки предназначены для переменных (строковых констант).

**1.1. В каких местах какие кавычки здесь можно поставить?**

**select * from public.users where public.users.id = 1**

```
    select * 
    from "public"."users" 
    where "public"."users"."id" = 1
```



###**2. В чём разница между UNION ALL и UNION, и какой из них работает быстрее?**
> UNION и UNION ALL объединяют результат двух разных SQL-запросов. 
  Они отличаются тем, как они обрабатывают дубликаты. 
  UNION выполняет DISTINCT в результирующем наборе, исключая любые повторяющиеся строки. 
  UNION ALL не удаляет дубликаты и поэтому работает быстрее, чем UNION.

**2.1. Какие могут быть практические способы применения одного и другого?**
> UNION  и UNION ALL используются для нахождения общего результата запроса из разных таблиц и полей (ивентов/товаров и так далее) с одинковым типом данных. Например объединение таблиц по разным годам или по разным типам товаров.


###**3. Каким образом в PostgreSQL вы преобразуете IPv4 адрес (хранится в типе inet) в десятичное представление IP-адреса?**


```
CREATE OR REPLACE FUNCTION ip2int(
  ip INET
) RETURNS BIGINT AS $$
WITH
  t AS (
    SELECT
      (col[1] :: INT * 256 ^ (row_number() OVER () - 1)) :: BIGINT AS num
    FROM REGEXP_MATCHES($1 :: VARCHAR, '(\d+)', 'g') AS t(col)
  )
SELECT
  sum(num) :: BIGINT AS ip_int
FROM t
$$ LANGUAGE SQL;
```



###**4. У вас есть таблицы carts (с корзинами пользователей) и cart_items (с товарами в корзине).**

**Структура таблиц:** 

**carts**

*id*  - первичный ключ

*state* - статус корзины, принимает значения pending, successful, canceled

*created_at* - timestamp создания корзины

*purchased_at* - timestamp оплаты корзины

*user_id* - внешний ключ к таблице users

 
**cart_items**

*id* - первичный ключ

*cart_id* - внешний ключ к таблице carts

*created_at* - timestamp добавления товара в корзину

*price* - стоимость товара

**Напишите SQL-запрос, который выведет по каждому пользователю дату и сумму первой покупки. Столбцы в выводе: user_id, date (в формате 30.01.2019), cart_amount.**
```
WITH first_orders AS (
  SELECT user_id,
         MIN(purchased_at) AS purchased_at
  FROM carts
  GROUP BY 1
                      )
SELECT user_id,
       purchased_at :: DATE AS date,
       sum(price) AS cart_amount
FROM first_orders fo
INNER JOIN carts
  ON fo.user_id = carts.user_id AND fo.purchased_at = carts.purchased_at
LEFT JOIN cart_items
  ON cart_id = carts.id
GROUP BY 1, 2
```



###**5. У вас есть таблица с транзакциями, поля в столбце payment_params в ней заполняются ответом по API от платёжной системы.**

{"date": 1561320934601, "amount": 566697, "bankInfo": {"bankCountryCode": "UNKNOWN", "bankCountryName": "<Неизвестно>"}, "currency": "643", "errorCode": "0", "actionCode": 0, "attributes": [{"name": "mdOrder", "value": "95f4b417-e544-7779-a1bf-77140015f31f"}], "terminalId": "20910698", "orderBundle": {"cartItems": {"items": [{"name": "Физика. Занимательная астрономия. 5–9 классы. Стандартный доступ", "itemCode": "courses/1005", "quantity": {"value": 1.0, "measure": "шт"}, "itemAmount": 62404, "positionId": "1", "itemCurrency": 643}, {"name": "Физика. Курс экспериментальной физики. 7 класс. Премиум доступ", "itemCode": "courses/1441", "quantity": {"value": 1.0, "measure": "шт"}, "itemAmount": 498059, "positionId": "2", "itemCurrency": 643}]}, "customerDetails": {"email": "email@mail.ru", "phone": "880055553535"}, "orderCreationDate": 1561320000}, "orderNumber": "462374", "orderStatus": 2, "authDateTime": 1561335081887, "errorMessage": "Успешно", "orderDescription": "", "paymentAmountInfo": {"paymentState": "DEPOSITED", "approvedAmount": 0, "refundedAmount": 0, "depositedAmount": 566697}, "merchantOrderParams": [{"name": "phone", "value": "88005553535"}, {"name": "initialAmount", "value": "573000"}, {"name": "ProductType", "value": "INSTALLMENT"}, {"name": "ProductID", "value": "10"}, {"name": "registerOrderId", "value": "b284708d-2f0e-4ce2-9d61-9190468dd362"}], "actionCodeDescription": ""}

**Опишите, как вы получите для каждой транзакции строку с названиями товаров, которые были в этой транзакции.**


In [0]:
import json
api_response = '{"date": 1561320934601, "amount": 566697, "bankInfo": {"bankCountryCode": "UNKNOWN", "bankCountryName": "<Неизвестно>"}, "currency": "643", "errorCode": "0", "actionCode": 0, "attributes": [{"name": "mdOrder", "value": "95f4b417-e544-7779-a1bf-77140015f31f"}], "terminalId": "20910698", "orderBundle": {"cartItems": {"items": [{"name": "Физика. Занимательная астрономия. 5–9 классы. Стандартный доступ", "itemCode": "courses/1005", "quantity": {"value": 1.0, "measure": "шт"}, "itemAmount": 62404, "positionId": "1", "itemCurrency": 643}, {"name": "Физика. Курс экспериментальной физики. 7 класс. Премиум доступ", "itemCode": "courses/1441", "quantity": {"value": 1.0, "measure": "шт"}, "itemAmount": 498059, "positionId": "2", "itemCurrency": 643}]}, "customerDetails": {"email": "email@mail.ru", "phone": "880055553535"}, "orderCreationDate": 1561320000}, "orderNumber": "462374", "orderStatus": 2, "authDateTime": 1561335081887, "errorMessage": "Успешно", "orderDescription": "", "paymentAmountInfo": {"paymentState": "DEPOSITED", "approvedAmount": 0, "refundedAmount": 0, "depositedAmount": 566697}, "merchantOrderParams": [{"name": "phone", "value": "88005553535"}, {"name": "initialAmount", "value": "573000"}, {"name": "ProductType", "value": "INSTALLMENT"}, {"name": "ProductID", "value": "10"}, {"name": "registerOrderId", "value": "b284708d-2f0e-4ce2-9d61-9190468dd362"}], "actionCodeDescription": ""}'
j = json.loads(api_response)
names = []
for item in j['orderBundle']['cartItems']['items']:
    names.append(item['name'])
names

['Физика. Занимательная астрономия. 5–9 классы. Стандартный доступ',
 'Физика. Курс экспериментальной физики. 7 класс. Премиум доступ']